# Stochastic Differential Equations, Quantum Phase Space, and Julia 

$$
\def\julia{\texttt{julia}}
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\E{{\rm e}}
\def\dx{\D x}
\def\dt{\D t}
\def\dz{\D z}
\def\C{{\mathbb C}}
\def\R{{\mathbb R}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqqquad{\qquad\qquad}
\def\qqfor{\qquad\hbox{for}\qquad}
\def\qqwhere{\qquad\hbox{where}\qquad}
\def\Res_#1{\underset{#1}{\rm Res}}\,
\def\sech{{\rm sech}\,}
\def\vc#1{{\mathbf #1}}
$$

Dr. Ashton Bradley
<br>
ashton.bradley@otago.ac.nz
<br>
http://amoqt.otago.ac.nz

## Workshop 3: Quantum Phase space
Numerical solution of SDEs for physicists

- [DifferentialEquations.jl](https://github.com/SciML/DifferentialEquations.jl): fast, adaptive stochastic integration
- Worked example: damped optical cavity 


References

- *The Quantum World of Ultracold Atoms and Light: Book I: Foundations of Quantum Optics*, C. W. Gardiner and P. Zoller, [Imperial College Press, London (2014)](https://www.worldscientific.com/worldscibooks/10.1142/p941)
- *Generalized P-representations in quantum optics*, P. D. Drumond and C. W. Gardiner, [J. Phys. A-Math. and Gen., __13__, 2353-2368 (1980)](http://iopscience.iop.org/article/10.1088/0305-4470/13/7/018/meta)
- [A deep introduction to julia](http://ucidatascienceinitiative.github.io/IntroToJulia/), Christopher Rackauckas
- [_Stochastic Lifestyle_](http://www.stochasticlifestyle.com/), Christopher Rackauckas
- *Dynamics and statistical mechanics of ultra-cold Bose gases using c-field techniques*,
P. B. Blakie, A. S. Bradley, M. J. Davis, R. J. Ballagh, and C. W. Gardiner, [Advances in Phyiscs 57, 363 (2008)](https://doi.org/10.1080/00018730802564254)

# SDE's in DifferentialEquations.jl

In [ ]:
using DifferentialEquations, Statistics, Plots, LaTeXStrings, Revise
gr(grid=false,legend=false,size=(400,200),titlefontsize=12)

## Ornstein-Uhlenbeck process

First example: 

$$du = -γ u dt + βdW(t)$$

Note that 
- for _additive noise_ it doesn't matter if we interpret this as an Ito $(I)$ or Stratonivich $(S)$ stochastic integral equation.
- In `DifferentialEquations.jl` there are different methods implemented for each discretization.

In [ ]:
γ = 0.1
β = 2.1
u0= 0.0
A(u,p,t) = -γ*u # drift
B(u,p,t) = β    # diffusion

t0 = 0.0; tf = 1.0; Nt = 200
dt = .0001
tspan = (t0,tf)
t = LinRange(tspan...,Nt)
prob = SDEProblem(A,B,u0,tspan)

Fixed step size, using the simplest Ito method: [Euler-Maruyama method](https://en.wikipedia.org/wiki/Euler%E2%80%93Maruyama_method)

In [ ]:
sol = solve(prob,EM(),dt=dt,saveat=t)
plot(sol,lw=1)

Fixed step size, using simplest Stratonivich method: [Euler-Heun method](https://en.wikipedia.org/wiki/Heun%27s_method)

In [ ]:
sol = solve(prob,EulerHeun(),dt=dt,saveat=t)
plot(sol,lw=1)

Adaptive integration using general method due to Lamba and Rackauckas. 

In [ ]:
sol = solve(prob,LambaEM(),saveat=t)
plot(sol,lw=1)

The trajectories look quite different! We are seeing the solution approach the steady state: adaptive solvers can stretch their legs and take bigger steps as dynamical timescales increase. Let's go back and check the timing. You can see this immediately translates to large gains in execution time. 

Historically, general methods are of low order of convergence, and performant adaptivity was not available. In my opinion, *general* adaptive methods that Rackauckas, Nie, and Lamba have been developing and sharing through $\julia$ are a total game changer.

See the article by [Rackauckas and Nie](https://www.aimsciences.org/article/doi/10.3934/dcdsb.2017133) for an in-depth discussion of rejection sampling with memory. 

### Documentation
For a list of all SDE methods see [SDE solvers](http://docs.juliadiffeq.org/latest/solvers/sde_solve.html) 

For a list of ODE methods, see [ODE solvers](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html#OrdinaryDiffEq.jl-1)

## Convergence and Monte Carlo and testing
Calling `solve` returns a single trajectory, allowing tests of _strong convergence_ for the integrator.

In Physics we are usually interested in sochastic averages, and this kind of convergence is called _weak convergence_. 

In `DifferentialEquations` the single trajectory problem is completely decoupled from the multi-trajectory problem. The latter is a _Monte Carlo problem_. There are separate libraries for solving MC problems, handling parallel execution, computing averages, etc. 

It is worth emphasizing here that $\julia$ package development is all done on github, and is ___test driven___. What this means is that `DifferentialEquations` has a vast battery of tests that the algorithms are automatically subjected to as part of any update to the code. In my experience, the tests are taken very seriously (nothing merges until tests pass), and I have never found any problems with integrators. 

That said, it can sometimes take a bit of digging to locate the integrator code and understand exactly what it is doing and how to pass the right parameter to achieve your desired level of accuracy or error estimator. The fundamental reason for this is that `DifferentialEquations.jl` is a large _meta-package_. 

Structure of a metapackage: [let's take a quick look at the code so you can see for yourself.](https://github.com/JuliaDiffEq/DifferentialEquations.jl)



## Monte Carlo problem

In [ ]:
monte_prob = MonteCarloProblem(prob) # adaptive, as that was last definition of prob

In [ ]:
sol = solve(monte_prob,trajectories=1000)

In [ ]:
summ = MonteCarloSummary(sol,0:0.01:1)
plot(summ,labels="Middle 95%")
summ = MonteCarloSummary(sol,0:0.01:1;quantiles=[0.25,0.75])
plot!(summ,labels="Middle 50%",legend=:topleft)
xlabel!("t");ylabel!("u(t)")
title!("Ornstein-Uhlenbeck process")

# Defining more complex problems

- Systems of coupled equations
- Real versus complex noise for quantum problems

In [ ]:
a = @ode_def_bare Drift begin
  du = -γ*u    
end γ β

In [ ]:
b = @ode_def_bare Diffusion begin
  du = β
end γ β

In [ ]:
function brownian(Ntraj)
# time 
ti = 0.
tf = 100.
Nt = 100
tspan = (ti,tf)
t = LinRange(ti,tf,Nt)
γ = 0.1; β = 0.5
p = (γ,β)
    
# initial condition
u0 = [6.0]
    
# define problem
prob = SDEProblem(a,b,u0,tspan,p)
    
alg = SRIW1() # higher order adaptive for diagonal noise
dt = 1/100
    
monte_prob = MonteCarloProblem(prob)


return solve(monte_prob,alg,saveat=t,trajectories=Ntraj)
end

In [ ]:
println("compile...")
brownian(1);

In [ ]:
Ntraj = 100
println("start...")
@time sol = brownian(Ntraj);

In [ ]:
plot(sol,lw=1) # few paths

In [ ]:
Ntraj = 5000 # let's get some clean statistics
println("start...")
@time sol = brownian(Ntraj);

In [ ]:
summ = MonteCarloSummary(sol)
plot(summ,labels="Middle 95%")
summ = MonteCarloSummary(sol;quantiles=[0.25,0.75])
plot!(summ,labels="Middle 50%",legend=:topright)
xlabel!("t");ylabel!("u(t)")
title!("Ornstein-Uhlenbeck process")

# Quantum Phase Space
My aim now is to walk you through the whole process for a simple problem. First a quick recap.

## Master Equation to SDE

We now wish to utilize the link between FPE and SDE to map the quantum dynamical problem described by the density matrix equation of motion 

$$\frac{\partial \rho}{\partial t} = -\frac{i}{\hbar}[H,\rho] + {\cal L}(\rho)\tag{open quantum system}$$

to the numerically easier problem of solving a set of coupled SDE's. 

For our purpose, the essential feature is to understand how the action of a creation or annihilation operator on the density matrix maps to an equivalent term in the FPE. 

From there, provided the FPE only has derivatives up to second order (strictly an FPE when true), and diffusion matrix is ___positive definite___ ($\textrm{det}D> 1$), there is a mapping of the quantum problem to an equivalent SDE.

We use the coherent state basis to effect the mapping

$$\dot{\rho}(t)\longrightarrow \partial_tP(\alpha,\alpha^*,t)\longrightarrow d\alpha(t)$$

Moments of $P$ give normally ordered operator averages:

<div class="alert alert-block alert-warning"><font color=blue>
$$\langle (a^\dagger)^na^m\rangle=\overline{((\alpha^*)^n\alpha^m)}_P$$
    </font></div>
    
The moments can be evaluated from stocahstic averaging over the paths of the SDE.

## P-function mappings
The simplest mapping is for the $P$-function, defined by the diagonal expansion in coherent states

$$\rho = \int d^2\alpha\; |\alpha\rangle\langle\alpha|P(\alpha,\alpha^*,t)$$

Provided such an expansion exists, we then have

$$a\rho = \int d^2\alpha\; |\alpha\rangle\langle\alpha|\alpha P(\alpha,\alpha^*,t)$$

and 

\begin{align}
a^\dagger\rho &= \int d^2\alpha\; P(\alpha,\alpha^*,t)\left(\alpha^*+\frac{\partial}{\partial \alpha}\right)|\alpha\rangle\langle\alpha|\notag\\
&=\int d^2\alpha\; |\alpha\rangle\langle\alpha|\left(\alpha^*-\frac{\partial}{\partial \alpha}\right)P(\alpha,\alpha^*,t)\notag
\end{align}

Notice here we have used integration by parts; we thus require that $P$ decays sufficiently quickly as $|\alpha|\to\infty$; there are (uncommon) situations where this is not valid, which will not concern us here.

Carrying out similar operations for $\rho a$, $\rho a^\dagger$, we establish the mapping of operators to differential operators acting on $P$

<div class="alert alert-block alert-warning"><font color=blue>
\begin{align}
a\rho &\longrightarrow \alpha P\notag\\
a^\dagger\rho &\longrightarrow \left(\alpha^*-\frac{\partial}{\partial \alpha}\right)P\tag{P-function mappings}\\
\rho a&\longrightarrow \left(\alpha-\frac{\partial}{\partial \alpha^*}\right)P\notag\\
\rho a^\dagger &\longrightarrow \alpha^* P\notag
\end{align}
</font></div>

- There are other important distributions, $Q$, $W$, and $+P$. The last two are the most useful in practical simulations.

# Damped harmonic oscillator
A well-known example including Hamiltonian and dissipative terms, a damped optical cavity. We take the Hamiltonian to be

$$H= \hbar\omega (a^\dagger a+\tfrac{1}{2})$$

## Master equation
Making the Born and Markov approximations, the density matrix time in the _Schroedinger picture_ is

\begin{align}
\dot{\rho}&=-\frac{i}{\hbar}[a^\dagger a,H]+\underbrace{\frac{\gamma}{2} (\bar n+1)\left[2a\rho a^\dagger - a^\dagger a\rho - \rho a^\dagger a \right]}_{\textrm{cavity losses: thermal and vacuum}}+\underbrace{\frac{\gamma}{2} \bar n\left[2a^\dagger\rho a - a a^\dagger\rho - \rho a a^\dagger \right]}_{\textrm{cavity gain: thermal}}
\end{align}

__Exercise:__ If you haven't worked with this kind of object before, use the cyclic properity of the trace, $\textrm{tr}(ABC)=\textrm{tr}(BCA)=\textrm{tr}(CAB)$, to show that the mean photon number $n=\langle a^\dagger a\rangle=\textrm{tr}(\rho a^\dagger a)$ evolves as

$$\frac{dn}{dt}=-\gamma (n - \bar n).$$ 

The steady state is simply $n_s=\bar n$, and the reservoir population at energy $\hbar \omega$ is 

$$\bar n=\frac{1}{e^{\beta \hbar\omega}-1}.\tag{Photon thermal state}$$

So the cavity mode and the environment reach equilibrium on a timescale or order $\sim 1/\gamma$, the inverse cavity loss rate.

## Mapping to P-function
Applying the mappings, taking care to preserve the order when acting on $\rho$ (action from the right, the order is reversed, i.e., apply the first operator to act on $\rho$ on the right as the first differential operator acting on $P$. Right-versus left can be chosen in either order, but you should apply all operator mappings on one side in order, then all on the other side), we find

\begin{align}\frac{\partial P}{\partial t}&=-\frac{\partial}{\partial\alpha}[\overbrace{(-i\omega-\tfrac{\gamma}{2})\alpha}^{\textrm{drift term for}\; d\alpha} P]-\frac{\partial}{\partial\alpha^*}[\overbrace{(i\omega-\tfrac{\gamma}{2})\alpha^*}^{\textrm{drift term for}\; d\alpha^*} P]\\
&+\underbrace{\frac{\gamma\bar n}{2}\frac{\partial^2}{\partial\alpha\partial \alpha^*}P+\frac{\gamma\bar n}{2}\frac{\partial^2}{\partial\alpha^*\partial \alpha}P}_{\textrm{ shows where entries are in diffusion matrix}}
\end{align}

## Equivalent SDE
The mapping from Ito SDE to FPE for an $n$-variable vector $\mathbf{x}$ of stochastic variables starts from the SDE

<div class="alert alert-block alert-warning"><font color=blue>
$$(I)d\mathbf{x}(t)=\mathbf{A}(\mathbf{x},t)dt+\mathbf{B}(\mathbf{x},t)d\mathbf{W}(t),$$
</font></div>

where $\mathbf{A}$ is $n$ dimensional ___drift vector___, $d\mathbf{W}(t)$ is an $m$-dimensional vector of real noises, and $\mathbf{B}$ is the $n\times m$ noise matrix. 

The condition that the $n\times n$ dimensional diffusion matrix $\mathbf{D}$ is positive-definite ensures that a factorization $\mathbf{D}=\mathbf{B}\mathbf{B}^T$ may be found. 

Using the same approach we took in lecture 2, it can be shown that this SDE is equivalent to the FPE

<div class="alert alert-block alert-warning"><font color=blue>
$$\frac{\partial p}{\partial t}=-\sum_i\partial_i[A_i(\mathbf{x},t)p]+\frac{1}{2}\sum_{i,j}\partial_i\partial_j([\mathbf{B}(\mathbf{x},t)\mathbf{B}^T(\mathbf{x},t)]_{ij}p)$$
    </font></div>
    

So we work backwards: find a factorization of the diffusion matrix. For convenience we take as our vector of unknowns 

$$\mathbf{x}= [\alpha\;\; \alpha^*]^T$$

The determinant of $\mathbf{D}$ can only be computed in terms of _real variables_, so some care should be taken. 

This choice gives the diffusion matrix (note the $1/2$ has gone!)

$$\mathbf{D}=\begin{bmatrix}
    0 & \gamma\bar n \\
    \gamma\bar n & 0 \\
\end{bmatrix},$$

that we factorize using the useful form (check this!)

$$\begin{bmatrix}
    0 & 1 \\
    1 & 0 \\
\end{bmatrix}=\frac{1}{\sqrt{2}}\begin{bmatrix}
    1 & i \\
    1 & -i \\
\end{bmatrix}\frac{1}{\sqrt{2}}\begin{bmatrix}
    1 & 1 \\
    i & -i \\
\end{bmatrix}$$

to find 

$$\mathbf{B}=\frac{\gamma\bar n}{\sqrt{2}}\begin{bmatrix}
    1 & i \\
    1 & -i \\
\end{bmatrix}.$$

Using the vector of real noises (Wiener increments)

$$d\mathbf{W}(t)=\begin{bmatrix}
    dw_1(t) \\
    dw_2(t) \\
\end{bmatrix}$$

we arrive the pair of SDE's

<div class="alert alert-block alert-warning"><font color=blue>
\begin{align}
(I)d\alpha&=(-i\omega-\gamma/2)\alpha dt+\sqrt{\gamma\bar n}\;dw(t)\\
(I)d\alpha^*&=(i\omega-\gamma/2)\alpha^* dt+\sqrt{\gamma\bar n}\;dw^*(t)
\end{align}
</font></div>

where the noise is now a _complex Wiener process_

$$dw(t)=\frac{1}{\sqrt{2}}(dW_1(t)+idW_2(t))$$

with correlations

\begin{align}
\langle dw(t)dw(t)\rangle&=0\\
\langle dw^*(t)dw(t)\rangle&=dt.
\end{align}

You can verify this using the properties of the standard real Wiener processes $dW_j(t)$, which have nonvanishing correlator $\langle dW_j(t)dW_k(t)\rangle=\delta_{jk}dt$.

Note that 
- In this case, the two equations are complex conjugates (same noise in each!)
- We only need to solve one SDE
- In `DifferentialEquations.jl` the default noise is of the same type as the fields, so this is the the default and we don't need any special noises.
- In $+P$ simulations we need _real_ noises, and want to work with _complex_ variables. In this case we would need to define our own noises; see [PhaseSpaceTools](https://github.com/AshtonSBradley/PhaseSpaceTools.jl) (a little package I have writting to do initial state sampling and provide some helpful methods for SDE's).

## Solving the SDE

In [ ]:
drift = @ode_def_bare Drift begin
  dα = (-im*ω-γ/2)*α    
  # dᾱ = (im*ω-γ/2)*ᾱ # in +P these would be independent
    end ω γ n̄

diffusion = @ode_def_bare Diffusion begin
  dα = sqrt(γ*n̄)
  # dᾱ = sqrt(γ*n̄) # in +P these would be independent
    end ω γ n̄

In [ ]:
function opticalcavity(Ntraj)
# time 
ti = 0.
tf = 50.
Nt = 100
t = LinRange(ti,tf,Nt)
tspan = (ti,tf)

ω = 1.0; γ = 0.1; n̄ = 10
p = (ω,γ,n̄)
    
# initial condition (must of the same type as solution! here an array of length 1)
a0 = [10.0+0.0*im]
    
    
prob = SDEProblem(drift,diffusion,a0,tspan,p)
alg = SRIW1() # higher order adaptive for diagonal noise
dt = 1/100
    
monte_prob = MonteCarloProblem(prob)

return solve(monte_prob,alg,saveat=t,trajectories=Ntraj)
end

In [ ]:
println("compile...")
opticalcavity(1);

In [ ]:
Ntraj = 5000  
println("start...")
@time sol = opticalcavity(Ntraj);

In [ ]:
# trajectories are first index
# modes are second index
# time is third index
α(j) = sol[j][1,:]

# trajectory average at each time, for each mode
n1 = zero(abs.(α(1)))
for j in 1:Ntraj
    n1 += abs2.(α(j))
end

n1 /= Ntraj; # divide by number of trajectories

We can check our results agains the analytic solution

$$n(t)=\bar{n}+e^{-\gamma t}(n_0-\bar{n})$$

In [ ]:
# we need the same parameters outside the function
ti = 0.
tf = 50.
Nt = 100
t = LinRange(ti,tf,Nt);

ω = 1.0
γ = 0.1
n̄ = 10 
n0 = 100
na(t) = n̄ + exp(-γ*t)*(n0-n̄)

In [ ]:
plot(t,na.(t),c=:red,lw=1,label="exact",legend=:topright)
plot!(t,n1,lw=6,c=:blue,alpha=.2,label="SDE")
plot!(t,one.(t)*n̄,s=:dash,label=L"\bar n")
xlims!(ti,tf);ylims!(0,1.1*n0)
ylabel!(L"\langle a^\dag a\rangle")
xlabel!(L"t");title!("Approach to thermal equilibrium")

## Remarks
- We have considered a simple and familiar example to highlight the key steps. You should now be able to work through any phase space mapping problem, find equivalent SDE's (approximately if truncation proves necessary), and solve them.
- The Wigner represtation has mappings that appear more symmetric. Moments of $W$, and hence stochastic averages over equivalent SDE's allow computation of *symmetric operator averages*, e.g. $\overline{(\alpha^*\alpha)}_W=\tfrac{1}{2}\langle a^\dagger a+a a^\dagger\rangle=\langle a^\dagger a\rangle+\tfrac{1}{2}$. This is a key distinction between $W$ and $P$, the latter giving *normally ordered operator averages*.
- This extra factor of $\tfrac{1}{2}$ is the source of vacuum noise in the truncated Wigner initial conditions that is absent from $P$-function methods.

# More Julia for physicists
- [ApproxFun](https://github.com/JuliaApproximation/ApproxFun.jl)
    - High precision calculations with Chebychev polynomials in 1D, 2D (inspired by [ChebFun](http://www.chebfun.org/) for Matlab)
- [Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl) 
    - allows you to define numbers with uncertainties, perform calculations involving them, and easily get the uncertainty of the result according to linear error propagation theory.
    - uses [PhysicalConstants](https://github.com/JuliaPhysics/PhysicalConstants.jl) and [UnitFul](https://github.com/ajkeller34/Unitful.jl)
- [Parameters](https://github.com/mauro3/Parameters.jl)
    - package to handle numerical-model parameters. 
    - keyword type constructors with default values for structs and NamedTuples,
    - unpacking and packing of composite types and dicts.
    - a great way to handle model parameter passing!

# From symbolic to efficient numerical evaluation using SymPy
In $\julia$ you can do computer algebra work in [SymPy](https://github.com/JuliaPy/SymPy.jl) (not a Mathematica replacement, but can do a lot of standard computer algebra in an jupyter environment that has a comparable user experience), and then create fast compiled code using the $\julia$ compiler, by using the function `lambdify`. 

Let's see a simple example, namely finding a jacobian. At this point there could be some namespace clashes, so before proceeding, restart and clear outputs using the kernel menu. 

In [1]:
using SymPy, BenchmarkTools
# (in this notebook we call SymPy methods by class to avoid 
# namespace clashes with other packages loaded above)

In [2]:
@syms x y

(x, y)

In [3]:
f(x,y) = sin(x)*exp(-y^2)

f (generic function with 1 method)

In [4]:
# build jacobian as a sym expression using an array comprehension
jac=[SymPy.diff(f(x,y), v1, v2) for v1 in [x,y], v2 in [x,y]]

2×2 Array{Sym,2}:
     -exp(-y^2)*sin(x)           -2*y*exp(-y^2)*cos(x)
 -2*y*exp(-y^2)*cos(x)  2*(2*y^2 - 1)*exp(-y^2)*sin(x)

In [5]:
?lambdify

search: lambdify



```
 lambdify(ex, vars; typ, fns, values, use_julia_code, invoke_latest)
```

Take a symbolic expression and return an anonymous `Julia` function

Converts from a SymPy object to an expression by walking the SymPy expression tree and converting each step. Then creates a function. The function arguments are based on `free_symbols`, and its ordering unless `vars` is specified directly.

  * `use_julia_code=false` will use SymPy's conversion to an expression, the default is `false`
  * `invoke_latest=true` calls `Base.invokelatest` to work around world age issues. This is th safe default, but setting to `false` will result in faster-executing functions.

Example:

```
@vars x y z
ex = x^2 * sin(x)
fn = lambdify(ex)
fn(pi)

ex = x + 2y + 3z
fn = lambdify(ex)
fn(1,2,3) # order is y,x,z

fn = lambdify(ex, (x,y,z))
fn(1,2,3)
```

!!! note
    Ideally, this would just be:


```
body = Meta.parse(julia_code(ex))
syms = Symbol.(free_symbols(ex))
fn = eval(Expr(:function, Expr(:call, gensym(), syms...), body))
```

Where the first line could also be `convert(Expr, ex)`. However, the `julia_code` method from sympy needs some attention.


In [6]:
# define a c-compiled function from symbolic expression
jac_lam=lambdify(jac,[x,y])

#86 (generic function with 1 method)

In [7]:
jac_lam(.1,.2)

2×2 Array{Float64,2}:
 -0.0959189  -0.382396
 -0.382396   -0.176491

# test against native julia

In [40]:
jac_native(x,y)=[-exp(-y^2)sin(x) -2y*exp(-y^2)cos(x);
            -2y*exp(-y^2)cos(x) 2(2y^2-1)exp(-y^2)sin(x)]

jac_native (generic function with 1 method)

In [41]:
jac_native(.1,.2)

2×2 Array{Float64,2}:
 -0.0959189  -0.382396
 -0.382396   -0.176491

In [73]:
@btime subs(jac[1],(x,.1),(y,.2)),subs(jac[2],(x,.1),(y,.2)),subs(jac[3],(x,.1),(y,.2)),subs(jac[4],(x,.1),(y,.2));

  421.011 μs (233 allocations: 7.42 KiB)


In [64]:
@btime jac_lam(.1,.2);

  5.755 μs (63 allocations: 1.41 KiB)


In [65]:
@btime jac_native(.1,.2);

  97.844 ns (5 allocations: 176 bytes)


So in this case lambdify is about 70 times faster than sympy, but still takes 6 times longer than native julia. 

In this simple example, we had the luxury of writing down a native julia expression, but the value of this approach comes when such a baseline is not available due to the complexity of expressions involved. In which case, a factor of 70 is a welcome improvement!